Câu2

In [3]:
import pandas as pd
import sqlite3
from scipy.stats import chi2_contingency

# Tạo DataFrame từ dữ liệu
data = {
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
}
df = pd.DataFrame(data)

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')

# Lưu DataFrame vào database
df.to_sql('car_samples', conn, index=False)

# Truy vấn SQL để chuyển đổi dữ liệu sang dạng quan hệ (tidy data)
query = """
SELECT 
    Day,
    'A' as Sample,
    A as Score
FROM car_samples
UNION ALL
SELECT 
    Day,
    'B' as Sample,
    B as Score
FROM car_samples
UNION ALL
SELECT 
    Day,
    'C' as Sample,
    C as Score
FROM car_samples
"""

# Lấy dữ liệu đã chuyển đổi
tidy_data = pd.read_sql(query, conn)

# Tạo bảng contingency (tần số chéo) cho kiểm định χ²
contingency_table = pd.crosstab(tidy_data['Day'], tidy_data['Sample'], values=tidy_data['Score'], aggfunc='sum')

# Thực hiện kiểm định χ²
chi2, p, dof, expected = chi2_contingency(contingency_table)

# In kết quả
print("=== Kết quả kiểm định χ² ===")
print(f"Giá trị χ²: {chi2:.4f}")
print(f"p-value: {p:.4f}")
print(f"Bậc tự do: {dof}")
print("\n=== Kết luận ===")
if p < 0.05:
    print("Có sự khác biệt đáng kể giữa các mẫu xe hoặc giữa các ngày thử nghiệm (p < 0.05).")
    print("Kết quả thử nghiệm phụ thuộc vào mẫu xe hoặc ngày thử nghiệm.")
else:
    print("Không có sự khác biệt đáng kể giữa các mẫu xe hoặc giữa các ngày thử nghiệm (p ≥ 0.05).")
    print("Kết quả thử nghiệm không phụ thuộc vào mẫu xe hoặc ngày thử nghiệm.")

# Đóng kết nối
conn.close()

=== Kết quả kiểm định χ² ===
Giá trị χ²: 0.8264
p-value: 0.9913
Bậc tự do: 6

=== Kết luận ===
Không có sự khác biệt đáng kể giữa các mẫu xe hoặc giữa các ngày thử nghiệm (p ≥ 0.05).
Kết quả thử nghiệm không phụ thuộc vào mẫu xe hoặc ngày thử nghiệm.


Câu3

In [1]:
import sqlite3
import pandas as pd

# Giả sử chúng ta có dữ liệu flights như sau
data = {
    'flight_id': [1, 2, 3, 4],
    'departure_time': [830, 1445, 935, 1230],
    # có thể có các cột khác
}

# Tạo DataFrame
df = pd.DataFrame(data)

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')

# Lưu DataFrame vào database
df.to_sql('flights', conn, index=False)

# Truy vấn SQL để chuyển đổi định dạng thời gian
query = """
SELECT 
    flight_id,
    departure_time AS original_time,
    -- Cách 1: Sử dụng hàm chuỗi để định dạng
    SUBSTR('0000' || departure_time, -4, 2) || ':' || SUBSTR('0000' || departure_time, -2, 2) AS formatted_time_1,
    
    -- Cách 2: Sử dụng phép toán số học (chia lấy phần nguyên và phần dư)
    (departure_time / 100) || ':' || (departure_time % 100) AS formatted_time_2,
    
    -- Cách 3: Xử lý trường hợp số có 3 chữ số (ví dụ: 945 -> 09:45)
    CASE 
        WHEN departure_time < 1000 THEN '0' || (departure_time / 100) || ':' || (departure_time % 100)
        ELSE (departure_time / 100) || ':' || (departure_time % 100)
    END AS formatted_time_3,
    
    -- Cách 4: Đảm bảo luôn có 2 chữ số cho phút
    (departure_time / 100) || ':' || SUBSTR('0' || (departure_time % 100), -2, 2) AS formatted_time_4
FROM 
    flights
"""

# Thực thi truy vấn và hiển thị kết quả
result = pd.read_sql(query, conn)
print(result)

# Đóng kết nối
conn.close()

   flight_id  original_time formatted_time_1 formatted_time_2  \
0          1            830            08:30             8:30   
1          2           1445            14:45            14:45   
2          3            935            09:35             9:35   
3          4           1230            12:30            12:30   

  formatted_time_3 formatted_time_4  
0            08:30             8:30  
1            14:45            14:45  
2            09:35             9:35  
3            12:30            12:30  


Câu5

In [2]:
import sqlite3

# Kết nối tới cơ sở dữ liệu SQLite (nếu chưa có, sẽ tạo mới)
conn = sqlite3.connect('patient.db')
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Patient (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        last_name TEXT,
        weight REAL,
        height REAL
    )
''')

# Chèn dữ liệu mẫu vào bảng Patient
# Giả sử một số bản ghi có last_name và weight giống hoặc gần giống nhau
data = [
    ('Smith', 70.5, 175.0),
    ('Smith', 71.0, 176.0),  # Gần giống bản ghi 1
    ('Johnson', 65.0, 180.0),
    ('Smith', 80.0, 170.0),  # last_name giống nhưng weight khác xa
    ('Johnson', 66.0, 181.0)  # Gần giống bản ghi 3
]

cursor.executemany('INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)', data)
conn.commit()

# Viết truy vấn SQL để so sánh các bản ghi
query = '''
SELECT 
    p1.id AS id1, 
    p1.last_name AS last_name1, 
    p1.weight AS weight1,
    p2.id AS id2, 
    p2.last_name AS last_name2, 
    p2.weight AS weight2
FROM Patient p1
JOIN Patient p2
ON p1.id < p2.id  -- So sánh từng cặp bản ghi, tránh lặp
WHERE 
    p1.last_name = p2.last_name  -- last_name giống nhau
    AND ABS(p1.weight - p2.weight) <= 2  -- weight chênh lệch không quá 2 đơn vị
'''

# Thực thi truy vấn
cursor.execute(query)
results = cursor.fetchall()

# Hiển thị kết quả
print("Các cặp bản ghi có thể đại diện cho cùng một người:")
for row in results:
    print(f"Bản ghi {row[0]} (last_name: {row[1]}, weight: {row[2]}) và "
          f"Bản ghi {row[3]} (last_name: {row[4]}, weight: {row[5]})")

# Đóng kết nối
conn.close()

Các cặp bản ghi có thể đại diện cho cùng một người:
Bản ghi 1 (last_name: Smith, weight: 70.5) và Bản ghi 2 (last_name: Smith, weight: 71.0)
Bản ghi 3 (last_name: Johnson, weight: 65.0) và Bản ghi 5 (last_name: Johnson, weight: 66.0)
